In [1]:
import pandas as pd

import numpy as np

import pymongo

import stringcase

from slugify import slugify

/home/johni/.pyenv/versions/3.7.4/envs/iara-ws-cnmp/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
MONGODB_URL = "mongodb://35.237.232.73:27017/"

client = pymongo.MongoClient(MONGODB_URL)

In [ ]:
database_name = "justica_estadual"

collection_name = "processos-tjsp"

processos = list(client[database_name][collection_name].find(limit=100))

print(f"Total de processos {len(processos)}")

In [ ]:
df = pd.read_excel("dados-pessoal-TJSP.xlsx", sep=";")
df.shape

In [ ]:
df.head()

In [ ]:
def buscar_forca_trabalho(seq_orgao):
    forca_trabalho = {}
    
    df_f = df[df["seq_orgao"] == seq_orgao]

    if len(df_f) == 0:
        print(f"Força de trabalho não encontrada para o orgão {seq_orgao}")
        return forca_trabalho
    
    for _, row in df_f.iterrows():
        descricao_variavel = row["Descrição da variável"]
        quantidade = int(row["Qtd"])
        
        descricao = stringcase.camelcase(slugify(descricao_variavel, separator='_'))
        
        forca_trabalho[descricao] = quantidade
        
    return forca_trabalho

def salvar_forca_trabalho(documento, forca_trabalho):
    collection = client[database_name][collection_name]

    _id = documento["_id"]

    collection.update_one(
        {
            "_id": _id 
        },
        {
            '$set': { 'dadosBasicos.orgaoJulgador.forcaTrabalho': forca_trabalho }
        }
    )
    
for documento in processos:
    orgao_julgador = documento["dadosBasicos"]["orgaoJulgador"]
    codigo_orgao = int(orgao_julgador["codigoOrgao"])
    forca_trabalho = buscar_forca_trabalho(codigo_orgao)
    salvar_forca_trabalho(documento, forca_trabalho)